<a href="https://colab.research.google.com/github/sgbaird/honegumi/blob/main/docs\generated_notebooks\ax\objective-single__model-GPEI__use_custom_gen-False__use_existing_data-True__use_constraint-False__use_categorical-False__use_custom_threshold-False.ipynb"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"></a>

In [ ]:
%pip install ax-platform

In [ ]:
import numpy as np
from ax.service.ax_client import AxClient, ObjectiveProperties


import pandas as pd

obj1_name = "branin"


def branin(x1, x2):
    y = float(
        (x2 - 5.1 / (4 * np.pi**2) * x1**2 + 5.0 / np.pi * x1 - 6.0) ** 2
        + 10 * (1 - 1.0 / (8 * np.pi)) * np.cos(x1)
        + 10
    )

    return y


# Define the training data
X_train = pd.DataFrame(
    [
        {
            "x1": -3.0,
            "x2": 2.0,
        },
        {
            "x1": 0.0,
            "x2": 7.0,
        },
        {
            "x1": 3.0,
            "x2": 5.0,
        },
        {
            "x1": 5.0,
            "x2": 0.0,
        },
        {
            "x1": 10.0,
            "x2": 10.0,
        },
    ]
)

# Calculate y_train using the objective function
y_train = [branin(row["x1"], row["x2"]) for _, row in X_train.iterrows()]


# Define the number of training examples
n_train = len(X_train)


ax_client = AxClient()
ax_client.create_experiment(
    parameters=[
        {"name": "x1", "type": "range", "bounds": [-5.0, 10.0]},
        {"name": "x2", "type": "range", "bounds": [0.0, 10.0]},
    ],
    objectives={
        obj1_name: ObjectiveProperties(minimize=True),
    },
)

# Add existing data to the AxClient
for i in range(n_train):
    ax_client.attach_trial(X_train.iloc[i].to_dict())
    ax_client.complete_trial(trial_index=i, raw_data=y_train[i])

for _ in range(10):
    parameters, trial_index = ax_client.get_next_trial()
    results = branin(
        parameters["x1"],
        parameters["x2"],
    )
    ax_client.complete_trial(trial_index=trial_index, raw_data=results)


best_parameters, metrics = ax_client.get_best_parameters()